In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import spacy
import re

## Data Inspection

In [ ]:
file_path = 'online_retail_II.xlsx'

df_1 = pd.read_excel(file_path, sheet_name='Year 2009-2010')
df_2 = pd.read_excel(file_path, sheet_name='Year 2010-2011')

df = pd.concat([df_1, df_2])

In [ ]:
df.head()

In [ ]:
df.info()
df.describe()

## Data Cleaning

In [ ]:
df1 = df[(df['Price'] > 0) & (df['Quantity'] > 0)]
df1 = df1[df1['InvoiceDate'].dt.year != 2009]
df1['Date'] = pd.to_datetime(df1['InvoiceDate']).dt.date

df1['Invoice'] = df1['Invoice'].astype(str)
df1 = df1[df1['Invoice'].str.startswith('C') == False]

df1 = df1.dropna(subset=['Description'])
df1['Description'] = df1['Description'].astype(str)

df1['StockCode'] = df1['StockCode'].astype(str)
df1['StockCode'] = df1['StockCode'].apply(str.upper)

df1 = df1[(df1['StockCode'].str.len() == 5) | (df1['StockCode'].str.len() == 6)]

In [ ]:
missing_values = df1.isna().sum()
print(missing_values)

In [ ]:
def remove_symbols(row):
    row = re.sub(r'[^a-zA-Z0-9\s]', ' ', row)
    row = ' '.join(row.split())
    return row

df1['Description'] = df1['Description'].apply(remove_symbols)

In [ ]:
df1['Description'] = df1['Description'].str.lower()

In [ ]:
df1.head()

In [ ]:
df1.info()

## Feature Extraction for Customisable Products

In [ ]:
products = df1[['StockCode', 'Description']].drop_duplicates()
products = products.reset_index(drop=True)
print(len(products))
products.head()

In [ ]:
vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = vectorizer.fit_transform(products['Description'])

cosine_sim = cosine_similarity(tfidf_matrix)
threshold = 0.75

products['Customisable'] = False
products['Variations'] = ''

In [ ]:
for i, description in enumerate(products['Description']):
    similar_indices = [j for j in range(len(products)) if (cosine_sim[i, j] > threshold and i!=j)]
    variations = []
    for j in similar_indices:
        temp = products['Description'].iloc[j]
        if (temp not in variations) and description != temp:
            variations.append(temp)
    if variations:
        products.at[i, 'Customisable'] = True
        products.at[i, 'Variations'] = variations

In [ ]:
customisable = products[products['Customisable']].reset_index(drop=True)
print(len(customisable))
customisable.head()

In [ ]:
customisable['Base Product'] = ''

for i, description in enumerate(customisable['Description']):
    variations = customisable.at[i, 'Variations']
    description_words = description.split()
    common = set(description_words)
    for variation in variations:
        variation_words = variation.split()
        common = common.intersection(set(variation_words))
    customisable.at[i, 'Base Product'] = ' '.join([word for word in description_words if word in common])

In [ ]:
customisable.head()

## Variation Extraction

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
customisable2 = customisable.copy()

In [ ]:
colors = [
    "red", "blue", "green", "yellow", "black", "white", "pink", "purple", "orange", "brown", "gray", "grey",
    "beige", "tan", "maroon", "cyan", "magenta", "crimson", "scarlet", "burgundy", "ruby", "rose", "wine",
    "cherry", "coral", "navy", "sky", "teal", "cobalt", "turquoise", "azure", "sapphire", "baby blue",
    "lime", "olive", "emerald", "mint", "sea green", "forest", "jade", "chartreuse", "golden", "lemon",
    "mustard", "amber", "sunflower", "canary", "jet", "charcoal", "ebony", "onyx", "matte black", "ink",
    "ivory", "snow", "cream", "pearl", "alabaster", "off-white", "bone", "blush", "fuchsia", "hot pink",
    "peach", "salmon", "bubblegum", "lavender", "lilac", "violet", "plum", "amethyst", "mauve", "orchid",
    "tangerine", "apricot", "rust", "chocolate", "coffee", "mahogany", "chestnut", "walnut", "bronze",
    "cocoa", "slate", "ash", "dove", "silver", "steel", "graphite", "sand", "khaki", "taupe", "buff",
    "camel", "almond", "gold", "silver", "copper", "brass", "rose gold", "platinum", "metallic",
    "pastel pink", "pastel blue", "pastel green", "pastel yellow", "pastel purple", "pastel orange",
    "neon pink", "neon green", "neon yellow", "neon blue", "neon orange", "terracotta", "sage", "moss",
    "umber", "ochre", "clay"
]

materials = [
    "metal", "wood", "plastic", "ceramic", "glass", "leather", "fabric", "stone", "marble", "wool",
    "cotton", "silk", "linen", "polyester", "nylon", "acrylic", "brass", "steel", "iron", "aluminum",
    "copper", "bronze", "bamboo", "rubber", "porcelain", "velvet", "suede", "lace", "canvas", "foam",
    "cardboard", "paper", "resin", "fiber", "synthetic", "polyurethane", "faux leather", "stainless steel",
    "plexiglass", "carbon fiber", "jute", "rattan", "hemp", "wicker", "spandex", "microfiber", "cashmere",
    "denim", "tweed", "charcoal", "glass fiber", "kevlar", "epoxy", "latex", "mesh", "plastic", "zinc"
]

sizes = [
    "extra small", "small", "medium", "large", "extra large", "extra extra large", "xx-small",
    "x-small", "small", "medium", "large", "x-large", "xx-large", "xxx-large",
    "xs", "s", "m", "l", "xl", "xxl", "xxxl", "xxxxl", "one size", "plus size", "petite",
    "tall", "regular", "slim fit", "relaxed fit", "oversized", "junior", "youth", "giant", "tiny"
    ]

festival_themes = [
    "christmas", "new year", "easter", "valentine's day", "halloween", "thanksgiving",
    "st. patrick's day", "independence day", "diwali", "hanukkah", "ramadan", "chinese new year",
    "mardi gras", "day of the dead", "oktoberfest", "back to school", "wedding", "birthday",
    "graduation", "anniversary", "spring", "summer", "fall", "winter"
]

patterns = [
    "stripes", "polka dots", "floral", "paisley", "plaid", "chevron", "herringbone",
    "geometric", "abstract", "camouflage", "tie-dye", "animal print", "leopard print",
    "zebra print", "snake print", "solid", "gradient", "marbled", "lace", "embroidered",
    "argyle", "batik", "ikat", "patchwork", "tartan", "watercolor", "checkered",
    "speckled", "textured", "holographic", "monogrammed", "embossed", "engraved"
]

shapes = [
    "round", "square", "rectangle", "oval", "triangle", "heart-shaped",
    "star-shaped", "hexagonal", "pentagonal", "circular", "cylindrical",
    "spiral", "cube", "pyramid", "diamond", "crescent", "cross", "arrow",
    "leaf-shaped", "flower-shaped", "teardrop", "drop-shaped", "octagonal",
    "butterfly-shaped", "wave", "cloud", "moon", "sun", "snowflake"
]

styles = [
    "vintage", "bohemian", "modern", "minimalist", "rustic", "gothic", "retro",
    "industrial", "mid-century modern", "art deco", "shabby chic", "farmhouse",
    "coastal", "nautical", "scandinavian", "victorian", "steampunk", "futuristic",
    "baroque", "asian-inspired", "mediterranean", "southwestern", "tropical",
    "contemporary", "urban", "eclectic", "colonial", "beach", "tribal",
    "organic", "romantic", "classical", "luxury", "glam", "zen", "avant-garde",
    "cyberpunk", "art nouveau", "country"
]

In [ ]:
def extract_variation(description, base_product):
    doc = nlp(description)
    variation_type = set()
    variation_detail = ""

    description_set = set(description.lower().split())
    base_product_set = set(base_product.lower().split())
    difference_set = description_set - base_product_set

    if not difference_set:
        return None, None, 0

    for token in doc:
        if token.text.lower() in colors:
            variation_type.add("colour")
            variation_detail += f"colour: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in sizes:
            variation_type.add("size")
            variation_detail += f"size: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in materials:
            variation_type.add("material")
            variation_detail += f"material: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in styles:
            variation_type.add("style")
            variation_detail += f"style: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in festival_themes:
            variation_type.add("festiveTheme")
            variation_detail += f"festiveTheme: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in patterns:
            variation_type.add("pattern")
            variation_detail += f"pattern: {token.text}, "
            break

    for token in doc:
        if token.text.lower() in shapes:
            variation_type.add("shape")
            variation_detail += f"shape: {token.text}, "
            break


    bundle_match = re.search(r'\b(set of|pack of)\s?\d+', description, re.IGNORECASE)
    if bundle_match:
        variation_type.add("bundle")
        variation_detail += f"bundle: {bundle_match.group()}, "

    difference_set -= set(colors) | set(sizes) | set(materials) | set(styles) | set(festival_themes) | set(patterns) | set(shapes)| set(bundle_match.group().split() if bundle_match else [])
    if difference_set:
        variation_type.add("miscellaneous")
        variation_detail += f"miscellaneous: {' '.join(difference_set)}"


    customisation_complexity = int(len(variation_type))

    variation_type = ', '.join(variation_type) if variation_type else None
    variation_detail = variation_detail.strip(', ') if variation_detail else None

    return variation_type, variation_detail, customisation_complexity

def extract_material(description):
    doc = nlp(description)
    material = None

    for token in doc:
        if token.text.lower() in materials:
            material = token.text
            break

    if not material:
        material = "polymer"

    return material

In [ ]:
customisable[['Variation Type', 'Variation Detail', 'Customisation Complexity']] = customisable.apply(lambda x: pd.Series(extract_variation(x['Description'], x['Base Product'])), axis=1)
customisable['Material'] = customisable['Description'].apply(extract_material)
customisable['Customisation Complexity'] = customisable['Customisation Complexity'].astype(int)

In [ ]:
customisable.head(20)

In [ ]:
customisable.tail(20)

In [ ]:
retail = df1.merge(customisable[['StockCode', 'Base Product', 'Variation Type', 'Variation Detail', 'Customisation Complexity', 'Material']], on='StockCode', how='inner')

In [ ]:
retail.head()

In [ ]:
retail.shape

In [ ]:
retail.to_excel("cleaned.xlsx")